In [36]:
import os
import sys
import time
import socket
import random
import ok


# file_dir = os.path.dirname(os.path.realpath(__file__))
# parent_dir = os.path.dirname(file_dir)
# sys.path.append(parent_dir)
dev_info = ok.okTDeviceInfo()
print("------ PipeTest Application fot Python API ----------")
print("Front Panel API version: {}".format(ok.okCFrontPanel_GetAPIVersionString()))
dev = ok.okCFrontPanel()
error_code = dev.OpenBySerial("")
if error_code != 0:
    print('Could not open the device.')
    print(error_code)
    sys.exit(1)



def initialize_FPGA(dev: ok.okCFrontPanel, bit_file_path):
    dev.GetDeviceInfo(dev_info)
    print("Found device: {}".format(dev_info.productName))
    dev.LoadDefaultPLLConfiguration()
    print('Device firmware version: {}.{}'.format(dev_info.deviceMajorVersion, dev_info.deviceMinorVersion))
    print('Device serial number: {}'.format(dev_info.serialNumber))
    print('Device ID: {}'.format(dev_info.productID))
    print(f"loading {bit_file_path} as bit file")
    if (dev.ConfigureFPGA(bit_file_path) != ok.okCFrontPanel.NoError):
        print("Fpga configuration failed.")
        return False
    print("success configuration")
initialize_FPGA(dev,"main_emulator.bit")
# initialize_FPGA(dev,"main_hardcoded.bit")

# ep_addr(8'h14), .ep_datain(ep20wireout));
# ep_addr(8'h15), .ep_datain(ep21wireout));
# ep_addr(8'h16), .ep_datain(ep22wireout));
# ep_addr(8'h18), .ep_datain(ep24wireout_readout));
# ep_addr(8'ha0), .ep_read(FIFO_read_from), .ep_blo



ep00wirein_address =  0x00
ep01wirein_address =  0x01
ep02wirein_address =  0x02
ep03wirein_address =  0x03
ep04wirein_address =  0x04
ep05wirein_address =  0x05
ep06wirein_address =  0x06
ep07wirein_address =  0x07
ep08wirein_address =  0x08
ep09wirein_address =  0x09
ep0awirein_address =  0x0A
ep0bwirein_address =  0x0B
ep0cwirein_address =  0x0C
ep0dwirein_address =  0x0D
ep0ewirein_address =  0x0E
ep0fwirein_address =  0x0F
ep10wirein_address =  0x10
ep11wirein_address =  0x11
ep12wirein_address =  0x12
ep13wirein_address =  0x13
ep14wirein_address =  0x14
ep15wirein_address =  0x15
ep16wirein_address =  0x16
ep17wirein_address =  0x17
ep18wirein_address =  0x18
ep19wirein_address =  0x19
ep1awirein_address =  0x1A
ep1bwirein_address =  0x1B              
ep1cwirein_address =  0x1C
ep1dwirein_address =  0x1D
ep1ewirein_address =  0x1E
ep1fwirein_address =  0x1F

# Trigger in endpoints
ep40trigin_address =  0x40
ep41trigin_address =  0x41

# wire out endpoints
ep20wireout_address = 0x20
ep21wireout_address = 0x21
ep22wireout_address = 0x22
ep23wireout_address = 0x23
ep24wireout_address = 0x24
ep25wireout_address = 0x25

ep3dwireout_address = 0x3D
ep3ewireout_address = 0x3E
ep3fwireout_address = 0x3F
# ep22wireout_address = 0x16
# ep24wireout_address = 0x18
epPipeOut_address   = 0xA0




# functions
def write_wr_in(dev,wr_addr,data,mask = 0xffffffff):
    dev.SetWireInValue(wr_addr, data,mask)
    dev.UpdateWireIns()  
def reset_sig(dev,ep_address):
    write_wr_in(dev,ep_address,0x00000001,mask=0x00000001)
    time.sleep(0.1)
    write_wr_in(dev,ep_address,0x00000000)
    time.sleep(0.1)
def write_length(dev,ep05wirein_address,length):
    write_wr_in(dev,ep05wirein_address,length)
    time.sleep(0.1)

def trigger_in(dev,trigin_address,index):
    dev.ActivateTriggerIn(trigin_address, index)

def trigger_spi(dev,trigin_address):
    dev.ActivateTriggerIn(trigin_address, 0)
    time.sleep(0.1)
def read_wire_out(dev,address):
    dev.UpdateWireOuts()
    ep20_val = dev.GetWireOutValue(address)
    return ep20_val
    # out_value = (ep20_val >> 8) & 0x7F  
    # out_debug = ep20_val & 0x7F
    # # print(f"Extracted ep{address}_val value: {ep20_val}")
    # # print(f"Extracted out value: {out_value}")
    # print(f"Extracted out_debug value: {out_debug}")
    # # return out_value, out_debug


def read_wire_out_print(dev, address, mode='dec'):
    dev.UpdateWireOuts()
    ep_val = dev.GetWireOutValue(address)
    if mode == 'hex':
        print(f"WireOut {hex(address)}: 0x{ep_val:04X}")
    else:  # default to decimal
        print(f"WireOut {hex(address)}: {ep_val}")
    return ep_val




# set_of_32 = 2
# length = 0x3F



# ep20wireout =    num_words_in_FIFO;
# ep21wireout =    { 16'b0, 12'b0, expander_ID_1, expander_detect, TTL_in_serial_exp, TTL_in_serial };
# ep22wireout =    { 16'b0, 15'b0, SPI_running };
# ep23wireout =    { 16'b0, TTL_in };
# ep24wireout =    { 16'b0, 14'b0, MMCM_prog_done, dataclk_locked };
# ep25wireout =    { 16'b0, 12'b0, board_mode };
# ep3ewireout =    { 16'b0, BOARD_ID };
# ep3fwireout =    { 16'b0, BOARD_VERSION };


def write_usb3_blocksize(dev,usb3_blocksize=0x00000000):
    write_wr_in(dev,ep1fwirein_address,usb3_blocksize)
    trigger_in(dev,ep40trigin_address,index=9)
def write_ddr_blocksize(dev,ddr_blocksize=0x00000000):
    write_wr_in(dev,ep1fwirein_address,ddr_blocksize)
    trigger_in(dev,ep40trigin_address,index=10)

def write_dataclk_MD(dev,Hex_00_M_D=0x00002A19):
    write_wr_in(dev,ep03wirein_address,Hex_00_M_D)
    trigger_in(dev,ep40trigin_address,index=0)      # MMCM_prog_trigger

def enable_streams(dev, enable_hex=0x00000000):
    write_wr_in(dev,ep14wirein_address,enable_hex) 

def write_DSP_settle_en(dev,Hex=0x00000000):
    write_wr_in(dev,ep1fwirein_address,Hex)
    trigger_in(dev,ep40trigin_address,index=6)  
    # trigger_in(dev,ep40trigin_address,index=7)      # MMCM_prog_trigger

def write_DAC_set_zero(dev):
    write_wr_in(dev,ep1fwirein_address,0x00000000)
    for i in range(32):
        trigger_in(dev,ep40trigin_address,index=i)  

def write_delay(dev,delay=0x00000001):
    write_wr_in(dev,ep04wirein_address,delay,mask=0x0000000f)

def write_max_time_samples(dev,time_step=0x00000100):
    print(f"Writing time step: {time_step}")
    write_wr_in(dev,ep01wirein_address,time_step)

def dac_stuff(dev):
    write_wr_in(dev,ep08wirein_address,0x00000000) 
    write_wr_in(dev,ep09wirein_address,0x00000000) 
    write_wr_in(dev,ep0awirein_address,0x00000000) 
    write_wr_in(dev,ep0bwirein_address,0x00000000) 
    write_wr_in(dev,ep0cwirein_address,0x00000000) 
    write_wr_in(    dev,ep15wirein_address,0x00000000) 
    write_wr_in(    dev,ep16wirein_address,0x00000000) 
    write_wr_in(    dev,ep17wirein_address,0x00000000) 
    write_wr_in(    dev,ep18wirein_address,0x00000000) 
    write_wr_in(    dev,ep19wirein_address,0x00000000) 
    write_wr_in(    dev,ep1awirein_address,0x00000000) 
    write_wr_in(    dev,ep1bwirein_address,0x00000000) 
    write_wr_in(    dev,ep1cwirein_address,0x00000000) 
    write_wr_in(    dev,ep1dwirein_address,0x00000000) 
    write_wr_in(    dev,ep1ewirein_address,0x00000000) 

# write_out wires status
# read_wire_out_print(dev, ep20wireout_address, mode='hex')
# read_wire_out_print(dev, ep21wireout_address, mode='hex')
# read_wire_out_print(dev, ep22wireout_address, mode='hex')
# read_wire_out_print(dev, ep23wireout_address, mode='hex')
# read_wire_out_print(dev, ep24wireout_address, mode='hex')
# read_wire_out_print(dev, ep25wireout_address, mode='hex')
# read_wire_out_print(dev, ep3ewireout_address, mode='hex')
# read_wire_out_print(dev, ep3ewireout_address, mode='dec')
read_wire_out_print(dev, ep3fwireout_address, mode='hex')
read_wire_out_print(dev, ep3fwireout_address, mode='dec')


print("FPGA is ready, starting INTAN configuration...")
#Trigger_in40 
reset_sig(              dev,    ep00wirein_address)
print("Reset done")
write_usb3_blocksize(   dev,    usb3_blocksize=0x00000000)
write_ddr_blocksize(    dev,    ddr_blocksize =0x00000000) 



write_dataclk_MD(       dev,    Hex_00_M_D    =0x00002A19) #

while (read_wire_out_print(dev, ep24wireout_address, mode='hex') != 0x00000003):
    print("Waiting for FPGA to be ready...")
    # time.sleep(0.5)

# DAC off, not used in this version
dac_stuff(dev)
write_DSP_settle_en(dev)
write_DAC_set_zero(dev)

print("Enabling streams...")
enable_streams( dev, enable_hex=0x0000000F)
write_delay(dev,delay=0x00000001)
write_max_time_samples(dev,time_step=0x00000100)



dev.ActivateTriggerIn(ep41trigin_address, 0)
# ep41trigin
print(f" Final EP00 wireout value: {read_wire_out_print(dev, ep00wirein_address, mode='hex')}")
print("FPGA configuration done, starting data acquisition...")
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')
read_wire_out_print(dev, ep20wireout_address, mode='dec')



------ PipeTest Application fot Python API ----------
Front Panel API version: 5.3.5
Found device: XEM7310-A200
Device firmware version: 1.30
Device serial number: 2436001CQA
Device ID: 44
loading main_emulator.bit as bit file
success configuration
WireOut 0x3f: 0x0001
WireOut 0x3f: 1
FPGA is ready, starting INTAN configuration...
Reset done
WireOut 0x24: 0x0003
Enabling streams...
Writing time step: 256
WireOut 0x0: 0x0000
 Final EP00 wireout value: 0
FPGA configuration done, starting data acquisition...
WireOut 0x20: 11428
WireOut 0x20: 17660
WireOut 0x20: 23932
WireOut 0x20: 50956
WireOut 0x20: 61371


61371

In [37]:
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")

WireOut 0x20: 79872
Data acquisition done.


In [38]:
import time

ep_addr = 0xA0  # Replace with your actual PipeOut endpoint address
ep20wireout_address = 0x20  # Replace with your actual WireOut address
chunk_size = 256  # Number of bytes to read per chunk
output_file = 'output_emulator.txt'  # Output file path

with open(output_file, 'w') as f:
    while True:
        # Update and read the WireOut value to determine bytes left
        dev.UpdateWireOuts()
        bytes_left = dev.GetWireOutValue(ep20wireout_address)
        print(f"WireOut {hex(ep20wireout_address)}: {bytes_left} bytes left")

        # Check if the remaining bytes are less than the chunk size
        if bytes_left < chunk_size:
            print(f"Final read: only {bytes_left} bytes left, which is less than the chunk size ({chunk_size} bytes).")
            break

        print(f"Reading {chunk_size} bytes from PipeOut endpoint {hex(ep_addr)}...")
        data = bytearray(chunk_size)
        ret = dev.ReadFromBlockPipeOut(ep_addr, chunk_size, data)

        if ret < 0:
            print(f"Error reading from Block Pipe Out. Error code: {ret}")
            break

        print("Data acquisition done.")

        for i in range(0, len(data), 4):
            chunk = data[i:i+4]
            if len(chunk) < 4:
                print("Incomplete 4-byte chunk encountered; skipping.")
                continue
            value = int.from_bytes(chunk, byteorder='little')  # Adjust 'little' or 'big' as needed
            # value = int.from_bytes(chunk, byteorder='big')  # Adjust 'little' or 'big' as needed
            line = f"0x{value:08X}"
            # line = f"Index {i//4:05d}: 0x{value:08X}"
            # print(line)
            f.write(line + '\n')

        # time.sleep(0.1)  # Optional: add delay between reads


WireOut 0x20: 79872 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 79744 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 79616 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 79488 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 79360 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 79232 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 79104 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 78976 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 78848 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Data acquisition done.
WireOut 0x20: 78720 bytes left
Reading 256 bytes from PipeOut endpoint 0xa0...
Dat

In [11]:
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")

WireOut 0x20: 74752
Data acquisition done.


In [4]:

set_of_32 = 2
length = 0x3F
ep_addr = epPipeOut_address 
# num_bytes = 32*4*set_of_32  
num_bytes = 1024  

print(f"Reading {num_bytes} bytes from PipeOut endpoint {hex(ep_addr)}...")
data = bytearray(num_bytes)
ret = dev.ReadFromBlockPipeOut(ep_addr, num_bytes, data)
read_wire_out_print(dev, ep20wireout_address, mode='dec')
print("Data acquisition done.")
if ret < 0:
    print("Error reading from Block Pipe Out. Error code:", ret,num_bytes,data)
else:
    print(data)

    for i in range(0, len(data), 4):
        chunk = data[i:i+4]
        if len(chunk) < 4:
            break  # Skip incomplete chunks at the end
        value = int.from_bytes(chunk, byteorder='little')  # Use 'big' if your data is big-endian
        print(f"Index {i//4:05d}: 0x{value:08X}")






Reading 1024 bytes from PipeOut endpoint 0xa0...
WireOut 0x20: 79360
Data acquisition done.
bytearray(b'\x00\x00S*\x00\x00\x138\x00\x00\xaa*\x00\x00\xa2\xd7\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03\x80\x00\x00\x04\x80\x00\x00\x01\x80\x00\x00\x02\x80\x00\x00\x03

In [ ]:

for i in range(0, len(data), 4):
    chunk = data[i:i+4]
    if len(chunk) < 4:
        break  # Skip incomplete chunks at the end
    value = int.from_bytes(chunk, byteorder='little')  # Use 'big' if your data is big-endian
    print(f"Index {i//4:05d}: 0x{value:08X}")


Index 00000: 0x2A530000
Index 00001: 0x38130000
Index 00002: 0x2AAA0000
Index 00003: 0xD7A20000
Index 00004: 0x00000000
Index 00005: 0x00000000
Index 00006: 0x80010000
Index 00007: 0x80010000
Index 00008: 0x80010000
Index 00009: 0x80010000
Index 00010: 0x80010000
Index 00011: 0x80010000
Index 00012: 0x80010000
Index 00013: 0x80010000
Index 00014: 0x80010000
Index 00015: 0x80010000
Index 00016: 0x80010000
Index 00017: 0x80010000
Index 00018: 0x80010000
Index 00019: 0x80010000
Index 00020: 0x80010000
Index 00021: 0x80010000
Index 00022: 0x80010000
Index 00023: 0x80010000
Index 00024: 0x80010000
Index 00025: 0x80010000
Index 00026: 0x80010000
Index 00027: 0x80010000
Index 00028: 0x80010000
Index 00029: 0x80010000
Index 00030: 0x80010000
Index 00031: 0x80010000
Index 00032: 0x80010000
Index 00033: 0x80010000
Index 00034: 0x80010000
Index 00035: 0x80010000
Index 00036: 0x80010000
Index 00037: 0x80010000
Index 00038: 0x80010000
Index 00039: 0x80010000
Index 00040: 0x80010000
Index 00041: 0x0

In [5]:
def process_bytearray(data, output_file=None):
    """
    Processes a bytearray, printing 32-bit values in hexadecimal format.
    Optionally writes the output to a specified file.

    :param data: The bytearray to process.
    :param output_file: Optional; path to the file to write the output.
    """
    lines = []
    for i in range(0, len(data), 4):
        chunk = data[i:i+4]
        if len(chunk) < 4:
            break  # Skip incomplete chunks at the end
        value = int.from_bytes(chunk, byteorder='little')  # Adjust 'little' or 'big' as needed
        line = f"Index {i//4:03d}: 0x{value:08X}"
        lines.append(line)

    # Print to console
    for line in lines:
        print(line)

    # Write to file if output_file is specified
    if output_file:
        try:
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write('\n'.join(lines))
            print(f"\nOutput successfully written to '{output_file}'")
        except IOError as e:
            print(f"An error occurred while writing to the file: {e}")


In [6]:
process_bytearray(data, output_file='output.txt')


Index 000: 0x2A530000
Index 001: 0x38130000
Index 002: 0x2AAA0000
Index 003: 0xD7A20000
Index 004: 0x00000000
Index 005: 0x00000000
Index 006: 0x80010000
Index 007: 0x80020000
Index 008: 0x80030000
Index 009: 0x80040000
Index 010: 0x80010000
Index 011: 0x80020000
Index 012: 0x80030000
Index 013: 0x80040000
Index 014: 0x80010000
Index 015: 0x80020000
Index 016: 0x80030000
Index 017: 0x80040000
Index 018: 0x80010000
Index 019: 0x80020000
Index 020: 0x80030000
Index 021: 0x80040000
Index 022: 0x80010000
Index 023: 0x80020000
Index 024: 0x80030000
Index 025: 0x80040000
Index 026: 0x80010000
Index 027: 0x80020000
Index 028: 0x80030000
Index 029: 0x80040000
Index 030: 0x80010000
Index 031: 0x80020000
Index 032: 0x80030000
Index 033: 0x80040000
Index 034: 0x80010000
Index 035: 0x80020000
Index 036: 0x80030000
Index 037: 0x80040000
Index 038: 0x80010000
Index 039: 0x80020000
Index 040: 0x80030000
Index 041: 0x80040000
Index 042: 0x80010000
Index 043: 0x80020000
Index 044: 0x80030000
Index 045: